In [1]:
"""Global module calls"""

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import sph_harm
from scipy.special import eval_genlaguerre as egl
from scipy.integrate import tplquad

In [2]:
"""Class(es)"""

class FCC:
    
    """
    Creates an object (whatever the user defines 'self' as) that has tied to it the array of points that are in an FCC
    lattice as well as the vectors needed that create that lattice from a superposition of basis vectors.
    
    ---
    Parameters:
    a, float: The value of the lattice constant along the x axis of the crystal geometry.
    b, float: The value of the lattice constant along the y axis of the crystal geometry.
    c, float: The value of the lattice constant along the z axis of the crystal geometry.
    basis_vectors, list/array: A list/array of the points that can be used to construct the 'lattice_vectors.'
    lattice_points, list/array: A list/array of the points that make up the lattice.
    lattice_vectors, list/array: A list/array of the vectors that can be used to construct the lattice.
    """
    
    def __init__(self, a, b, c):
        
        """
        Initializes all properties of the FCC crystal lattice object tied to 'self.'
        
        # a, float: The value of the lattice constant along the x axis of the crystal geometry.
        # b, float: The value of the lattice constant along the y axis of the crystal geometry.
        # c, float: The value of the lattice constant along the z axis of the crystal geometry.
        
        ---
        Returns: Nothing since this function initializes the object and the properties of the object.
        
        """
        
        ## All of the initializations of the object 'self.'
        self.a = a # The length of the side of the lattice that is attached conventionally to the Cartesian x axis in a 3D plot.
        self.b = b # The length of the side of the lattice that is attached conventionally to the Cartesian y axis in a 3D plot.
        self.c = c # The length of the side of the lattice that is attached conventionally to the Cartesian z axis in a 3D plot.
        self.cryst_basis = np.array([[a/2, b/2, 0], [a/2, 0, c/2], [0, b/2, c/2]]) # This is a basis of not-necessarily-orthogonal vectors that can be used to construct the lattice.
        self.R_j = np.array(self.fcc()) # All of the points of the FCC crystal lattice that can conveniently be used as vectors.
        self.bcc = np.array(self.bcc()) # All of the points of the BCC crystal lattice (the FCC reciprocal lattice) that can be conveniently used as vectors.

    @classmethod
    def lengthen(self, *lat_con):
        
        """
        Lengthens the values of a, b and/or c by unpacking them from 'lat_con' and then resetting the values instantiated 
        by '__init__'.
        
        ---
        Parameters:
        self, object: The object 'self.'
        lat_con, int: A lattice constant.
        
        ---
        Returns: Nothing.
        """
        
        ## Creating a list of lattice constant values for ease of maneuvering through all their values.
        lat_arr = [] # Initializing an array of the lattice constant values.
        for i in lat_con: # Iterating through the values to create said array. This is for the case(s) when less than three or more than three values are passed to this function.
            lat_arr.append(i)
            
        ## Several if statements to make the changes based on how many values are provided to the function 'lengthen.'
        if len(lat_arr) == 0:
            print('No constants were provided so none of the constants will be changed.')
        elif len(lat_arr) == 1: # Changes to only one parameter.
            self.a = lat_arr[0]
        elif len(lat_arr) == 2: # Changes to only two parameters.
            self.a = lat_arr[0]
            self.b = lat_arr[1]
        elif len(lat_arr) == 3: # Changes to all three parameters.
            self.a = lat_arr[0]
            self.b = lat_arr[1]
            self.c = lat_arr[2]
        else: # Changes to too many parameters.
            print('You have provided more constants than there are dimensions of this geometry so nothing can be changed.')

    def fcc(self):
        
        """
        Creates a list of the points on the Face Centered Cubic crystal lattice.
        
        ---
        Parameters:
        self, object: The object itself.
        
        ---
        Returns: A list of vectors in the FCC lattice in spherical coordinates.
        """
                
        ## Initializing the matrix with the Cartesian values of the lattice points.
        # Initial Cartesian points.
        cart = [[0, 0, 0], [self.a, 0, 0], [0, self.b, 0], [0, 0, self.c], [self.a, self.b, 0], [self.a, 0, self.c], [0, self.b, self.c], [self.a, self.b, self.c],
                         [self.a/2, self.b/2, 0], [self.a/2, 0, self.c/2], [0 , self.b/2, self.c/2], [self.a/2, self.b/2, self.c], [self.a/2, self.b, self.c/2], [self.a, self.b/2, self.c/2]]
        
        ## Creating the Spherical versions of the lattice points.
        spherical = []
        for i in range(len(cart)):
            r = np.sqrt(cart[i][0] ** 2 + cart[i][1] ** 2 + cart[i][2] ** 2)
            try:
                if cart[i][2] != 0:
                    theta = np.arctan(np.sqrt(cart[i][0] ** 2 + cart[i][1] ** 2) / cart[i][2])
                else:
                    theta = np.pi / 2
            except:
                print('There was an unaccounted for error and therefore theta does not have a value.')
            try:
                if cart[i][0] != 0:
                    phi = np.arctan(cart[i][1] / cart[i][0])
                else:
                    phi = np.pi / 2
            except:
                print('There was an unaccounted for error and therefore phi does not have a value.')
            spherical.append(np.array([r, theta, phi]))
        
        return spherical
    
    def bcc(self):
        
        """
        Creates a list of the points in the reciprocal lattice of the original FCC lattice. Thankfully, no math has to be
        done because this is simply a BCC lattice. I am most likely incorrect but these could serve as k values for the
        Brillouin Zone.
        
        ---
        Parameters:
        self, object: The object itself.
        
        ---
        Returns: A list of the vectors in the BCC lattice in spherical coordinates.
        """
        
        ## Initializing the matrix with the Cartesian values of the lattice points.
        # Initial Cartesian points.
        cart = [[0, 0, 0], [self.a, 0, 0], [0, self.b, 0], [0, 0, self.c], [self.a, self.b, 0] ,[self.a, 0, self.c], [0, self.b, self.c], [self.a, self.b, self.c], [self.a/2, self.b/2, self.c/2]]
        
        ## Creating the Spherical versions of the lattice points.
        spherical = []
        for i in range(len(cart)):
            r = np.sqrt(cart[i][0] ** 2 + cart[i][1] ** 2 + cart[i][2] ** 2)
            try:
                if cart[i][2] != 0:
                    theta = np.arctan(np.sqrt(cart[i][0] ** 2 + cart[i][1] ** 2) / cart[i][2])
                elif cart[i][2] == 0:
                    theta = np.pi / 2
            except:
                print('There was an unaccounted for error and therefore theta does not have a value.')
            try:
                if cart[i][0] != 0:
                    phi = np.arctan(cart[i][1] / cart[i][0])
                elif cart[i][0] == 0:
                    phi = np.pi / 2
            except:
                print('There was an unaccounted for error and therefore phi does not have a value.')
            spherical.append(np.array([r, theta, phi]))
        
        return spherical

In [3]:
"""Functions"""

def wavefunction(n, l, m, Z, r, theta, phi):
    
    """
    Creates the hydrogenic wavefunctions of the atoms.
    
    ---
    Parameters:
    n, int: The value of the principle quantum number.
    l, int: The value of the angular momentum quantum number.
    m, int: The value of the z-component of the angular momentum quantum number.
    Z, int: The number of protons in the nucleus of the atom.
    r, array: An array of the values that the positional variable can take.
    theta, array: An array of the values that the azimuthal angle can take.
    phi, array: An array of the the values that the altudinal angle can take.
    
    ---
    Returns: An array of the values of the wavefunction Psi_nlm.
    """
            
    ## Calculating the radial component of the wavefunction. Calling it rnl for brevity.
    rnl = np.exp(-((Z*r) / n)) * (((2*Z*r) / n) ** l) * egl((n-l-1), (2*(l+1)), ((2*Z*r) / n))
    
    ## Calculating the angular component of the wavefunction. Calling it ylm for brevity.
    ylm = sph_harm(m, l, theta, phi)
    
    ## Forming the full unnormalized Hydrogenic wavefunction.
    psinlm = rnl * ylm

    return psinlm

def atm_E(n, Z):
    
    """
    Creates the energy value of the Hydrogenic atomic wavefunctions.
    
    ---
    Parameters:
    n, int: The value of the principle quantum number.
    Z, int: The number of protons in nucleus of the atom.
    
    ---
    Returns: A scalar which represents the value of the energy of the atomic orbitals.
    """
        
    return ((-13.6 * Z ** 2) / (n ** 2))

def overlap(k, R_j, n, l, m, Z, r, theta, phi):
    
    """
    Creates an array of the values of the overlap integral multiplied by the summation term.
    
    ---
    Parameters:
    k, list/array: A (An) list (array) of the Brillouin Zone (k-space) values of interest.
    R_j, list/array: A (An) list (array) of the lattice points in the real space crystal lattice.
    n, int: The value of the principle quantum number.
    l, int: The value of the angular momentum quantum number.
    m, int: The value of the z-projection of the angular momentum quantum number.
    Z, int: The number of protons in nucleus of the atom.
    
    ---
    Returns: A (An) list (array) of the values for the overlap term to be plotted in k-space.
    """
        
    ## Creating the overlap array
    overlap_term = []
    
    ## Going through the R_j and k terms.
    for k_index in range(len(k)):
        for i in range(len(R_j)):
            overlap_term.append(np.exp(-1j * k[k_index] * R_j[i]) * tplquad(wavefunction(n, l, m, Z, r, theta, phi) *
                           wavefunction(n, l, m, Z, r - R_j[i][0], theta, phi), 0, np.inf, 0, 2*np.pi, 0, np.pi))
        
    return overlap_term

def plotting(n, l, m, k, E):
    
    """
    Creates a plot of the different energy bands.
    
    ---
    Parameters:
    n, int: The value of the principle quantum number.
    l, list: A list of all possible values of angular momentum quantum number.
    m, list: A list of all the possible values of the z-component of the angular momentum quantum number.
    k, array: An array of the k values on which the energy is plotted. Will be changed later to take certain paths in the Brillouin Zone.
    E, array: An array of the energy of the crystal.
    
    ---
    Returns: Nothing.
    """
    
    fig, ax = plt.subplots(l, m, sharex=True, sharey=True)
    for i in range(l):
        for j in range(m):
            ax[i, j].plot(k, E)
    plt.show()       

In [4]:
"""Calls."""

## Initializing the object 'self' as 'fcc.'
a = b = c = 0.532 # Assuming a cubic structure instead of tetragonal.
fcc = FCC(a, b, c)

## Creating the spherical positional arrays. This is commented out because I may not need these anymore.
r = np.array([i for i in range(100)])
theta = np.array([(2 * np.pi * i) / 100 for i in range(100)])
phi = np.array([(np.pi * i) / 100 for i in range(100)])

## Defining all the values.
# Creating the values in the Brillouin Zone.
k_vals = fcc.bcc

# Defining the number of protons in the nucleus of the atom and making sure that it's an integer.
Z = ''
while type(Z) is not int:
    Z = input('How many protons are there in your atom? ')
    try:
        Z = int(Z)
    except:
        print('Please try again as this is not a number.')
        
# Defining the principle quantum number and making sure it's an integer.
n = ''
while type(n) is not int:
    n = input('What is the value of the principle quantum number? ')
    try:
        n = int(n)
    except:
        print('Please try again as this is not a number.')

# Defining the angular momentum quantum number as defined from the n value.
l = np.array(list(range(0, n)))

# Defining the z component of the angular quantum number as defined from the l value.
m = np.array([np.array(list(range(-i, i+1))) for i in l if i != 0])

## Defining the actual overlap term.
overlap_term = overlap(k_vals, fcc.R_j, n, l[0], m[0][0], Z, r, theta, phi)

How many protons are there in your atom? 23
What is the value of the principle quantum number? 6


ValueError: invalid callable given